In [2]:
# Install dependencies
!pip install -q -r requirements.txt

In [3]:
import os
import numpy as np
import pandas as pd
from scipy.spatial import distance
import notebook_utils.notebook_helpers as utils
from cuopt import routing
import cudf

# Benchmark Gehring & Homberger
## Capacitated Vehicle Routing Problem with Time Windows (CVRPTW)

While other notebooks such as [cvrptw_service_team_routing.ipynb](cvrptw_service_team_routing.ipynb) focus on the cuOpt API and high level problem modeling, here we focus on performance.

cuOpt offers a unique benefit over other solver_settingss, specifically, time to solution.  In addition to achieving world class accuracy, cuOpt also produces these solutions in a time frame that allows for re-optimization in dynamic environments and rapid iteration over possible problem configurations.

Here we are demonstrating this performance on a large popular academic [dataset by Gehring & Homberger](https://www.sintef.no/projectweb/top/vrptw/homberger-benchmark/).  These problems are well studied and used as the basis for comparison for VRP research and product offerings. The particular instance we will test with is from the group of largest (1000 location) problems.  Each problem instance has an associated best known solution, the one we will measure against is shown below

**API Reference**: [cuOpt Documentation](https://docs.nvidia.com/cuopt)

In [4]:
homberger_1000_file = 'notebook_utils/data/C1_10_1.TXT'

best_known_solution = {
    "n_vehicles": 100,
    "cost": 42478.95
}

### Problem Data
The data for this problem instance are provided via text file.  cuOpt has a utility function available specifically for the Gehring & Homberger benchmark which converts the problem into the components required by cuOpt.

In [6]:
orders, vehicle_capacity, n_vehicles = utils.create_from_file(homberger_1000_file)
n_locations = orders["demand"].shape[0]-1
print("Number of locations          : ", n_locations)
print("Number of vehicles available : ", n_vehicles)
print("Capacity of each vehicle     : ", vehicle_capacity)
print("\nInitial Orders information")
print(orders)

/home/nfs/aanandh/cuoptres/last_mile_delivery/notebook_utils/notebook_helpers.py:366: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(row, index=[0])], ignore_index=True)


Number of locations          :  1000
Number of vehicles available :  250
Capacity of each vehicle     :  200

Initial Orders information
     vertex  xcord  ycord demand earliest_time latest_time service_time  \
0         0  250.0  250.0      0             0        1824            0   
1         1  387.0  297.0     10           200         270           90   
2         2    5.0  297.0     10           955        1017           90   
3         3  355.0  177.0     20           194         245           90   
4         4   78.0  346.0     30           355         403           90   
...     ...    ...    ...    ...           ...         ...          ...   
996     996  330.0  242.0     30           627         671           90   
997     997  332.0  249.0     30            82         144           90   
998     998  375.0   80.0     10           550         598           90   
999     999   94.0  235.0     20           227         266           90   
1000   1000  287.0  144.0     20      

# Initialize cuOpt Problem Model

In [7]:
# Create a routing model with the necessary locations and vehicles
data_model = routing.DataModel(n_locations + 1, n_vehicles)

### Cost Matrix

In [8]:
coords = list(zip(orders['xcord'].to_list(),
                  orders['ycord'].to_list()))

cost_matrix = distance.cdist(coords, coords, 'euclidean')
cost_matrix_df = cudf.DataFrame(cost_matrix.astype(np.float32))

### Set Cost Matrix

In [9]:
# Add the distance matrix as our cost matrix
data_model.add_cost_matrix(cost_matrix_df)

### Set Fleet Data

In [10]:
# All vehicles start and end at the depot (location 0)
veh_start_locations = cudf.Series([0] * n_vehicles)
veh_end_locations = cudf.Series([0] * n_vehicles)
data_model.set_vehicle_locations(veh_start_locations, veh_end_locations)

# Set vehicle capacities
vehicle_capacities = cudf.Series([vehicle_capacity] * n_vehicles, dtype=np.int32)

### Set Demand and Capacity

In [11]:
# Convert demand to cudf Series
location_demand = cudf.Series(orders['demand'].values, dtype=np.int32)

# Add demand and capacity dimension
data_model.add_capacity_dimension("demand", location_demand, vehicle_capacities)

### Set Time Windows

In [12]:
# Set time windows for locations
earliest_times = cudf.Series(orders['earliest_time'].values, dtype=np.int32)
latest_times = cudf.Series(orders['latest_time'].values, dtype=np.int32)
data_model.set_order_time_windows(earliest_times, latest_times)

# Set service times
service_times = cudf.Series(orders['service_time'].values, dtype=np.int32)
data_model.set_order_service_times(service_times)

AttributeError: 'DataModel' object has no attribute 'set_location_time_windows'

### Helper functions to solve and process the output

In [ ]:
def solution_eval(vehicles, cost, best_known_solution):
    
    print(f"- cuOpt provides a solution using {vehicles} vehicles")
    print(f"- This represents {vehicles - best_known_solution['n_vehicles']} more than the best known solution")
    print(f"- Vehicle Percent Difference {(vehicles/best_known_solution['n_vehicles'] - 1)*100}% \n\n")
    print(f"- In addition cuOpt provides a solution cost of {cost}") 
    print(f"- Best known solution cost is {best_known_solution['cost']}")
    print(f"- Cost Percent Difference {(cost/best_known_solution['cost'] - 1)*100}%")

### Get Optimized Results

Update solver config and test different run-time 

**1 Minute Time Limit**

Note: due to the large amount of data network transfer time can exceed the requested solve time.


In [ ]:
# Create solver settings with 60 second time limit
solver_settings = routing.SolverSettings()
solver_settings.set_time_limit(60.0)

# Solve the problem
solution = routing.Solve(data_model, solver_settings)

# Get solution metrics
if solution.get_status() == 0:  # Success
    num_vehicles = solution.get_vehicle_count()
    solution_cost = solution.get_total_objective()
    print(f"Solution found with status: {solution.get_status()}")
    print(f"Number of vehicles used: {num_vehicles}")
    print(f"Total solution cost: {solution_cost}")
else:
    print(f"Failed to find a solution. Status: {solution.get_status()}")

In [ ]:
# Evaluation:
if solution.get_status() == 0:  # Success
    solution_eval(num_vehicles, solution_cost, best_known_solution)

**2 Minute Time Limit**

In [ ]:
# Create solver settings with 120 second time limit
solver_settings = routing.SolverSettings()
solver_settings.set_time_limit(120.0)

# Solve the problem
solution = routing.Solve(data_model, solver_settings)

# Get solution metrics
if solution.get_status() == 0:  # Success
    num_vehicles = solution.get_vehicle_count()
    solution_cost = solution.get_total_objective()
    print(f"Solution found with status: {solution.get_status()}")
    print(f"Number of vehicles used: {num_vehicles}")
    print(f"Total solution cost: {solution_cost}")
else:
    print(f"Failed to find a solution. Status: {solution.get_status()}")

In [ ]:
# Evaluation:
if solution.get_status() == 0:  # Success
    solution_eval(num_vehicles, solution_cost, best_known_solution)


SPDX-FileCopyrightText: Copyright (c) 2024 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
SPDX-License-Identifier: MIT
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.